# SVD 기반 협업 필터링(Collaborative Filtering)

In [ ]:
# userID를 인자로 받으면 이 사람이 안 본 동영상 중에 가장 예상 평점 높은것의 movieID 반환

In [1]:
from surprise import Reader, Dataset, SVD, accuracy
import pandas as pd

In [2]:
def createRatingMatrix(filename):   
    data_frame = pd.read_csv(filename, sep="::", usecols = [0, 1, 2], names = ['userID', 'movieID', 'rating'], engine = 'python')
    return data_frame

In [3]:
def createMovieMatrix(filename):   
    data_frame = pd.read_csv(filename, sep="::", usecols = [0, 1, 2], names = ['movieID', 'title', 'genres'], engine = 'python')
    return data_frame

In [4]:
def get_unseen_movies(ratings, movies, userID):
     # 특정 userId가 평점을 매긴 모든 영화 리스트
    seen_movies = ratings[ratings['userID']== userID]['movieID'].tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    total_movies = movies['movieID'].tolist()
      
    # 한줄 for + if문으로 안 본 영화 리스트 생성
    unseen_movies = [ movie for movie in total_movies if movie not in seen_movies]
    
    return unseen_movies

In [5]:
def recomm_best_movie_by_svd(algo, userID, unseen_movies):
    
    # 아직 보지 않은 영화의 예측 평점: prediction 객체 생성
    predictions = []    
    for movieID in unseen_movies:
        predictions.append(algo.predict(userID, movieID))
    
    # 리스트 내의 prediction 객체의 est(예측값)를 기준으로 내림차순 정렬
    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True) # key에 리스트 내 객체의 정렬 기준을 입력
    
    # 상위 top-1개의 prediction 객체
    top_prediction = predictions[0]

    movie_id = int(top_prediction.iid)
    
    return movie_id

In [6]:
user_ratings = createRatingMatrix('../ml-1m/ratings.dat')
movie_info = createMovieMatrix('../ml-1m/movies.dat')

In [7]:
reader = Reader()
# load_from_df사용해서 데이터프레임을 데이터셋으로 로드
# 인자에 userid-itemid-ratings 변수들이 포함된 데이터프레임형태로 넣어주면 됨!
data = Dataset.load_from_df(user_ratings[['userID','movieID','rating']],reader=reader)

In [8]:
algo = SVD(n_factors=20)
# 전체 data 대상 학습 진행
trainset = data.build_full_trainset()
algo.fit(trainset)

In [9]:
# 1번 유저에 대해 계산
unseen_movies = get_unseen_movies(user_ratings, movie_info, 1)

In [10]:
#1번 유저에 대해 계산
recomm_best_movie_by_svd(algo, 1, unseen_movies)

953